In [21]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import folium

In [4]:
# Install Packages
# pip install folium

# Reading the dataset

In [5]:
cat = pd.read_csv('archive/covid_impact_on_airport_traffic.csv') # cat == covid_airport_traffic
cat.head()

,AggregationMethod,Date,Version,AirportName,PercentOfBaseline,Centroid,City,State,ISO_3166_2,Country,Geography
0,Daily,2020-04-03,1.0,Kingsford Smith,64,POINT(151.180087713813 -33.9459774986125),Sydney,New South Wales,AU,Australia,"POLYGON((151.164354085922 -33.9301772341877, 1..."
1,Daily,2020-04-13,1.0,Kingsford Smith,29,POINT(151.180087713813 -33.9459774986125),Sydney,New South Wales,AU,Australia,"POLYGON((151.164354085922 -33.9301772341877, 1..."
2,Daily,2020-07-10,1.0,Kingsford Smith,54,POINT(151.180087713813 -33.9459774986125),Sydney,New South Wales,AU,Australia,"POLYGON((151.164354085922 -33.9301772341877, 1..."
3,Daily,2020-09-02,1.0,Kingsford Smith,18,POINT(151.180087713813 -33.9459774986125),Sydney,New South Wales,AU,Australia,"POLYGON((151.164354085922 -33.9301772341877, 1..."
4,Daily,2020-10-31,1.0,Kingsford Smith,22,POINT(151.180087713813 -33.9459774986125),Sydney,New South Wales,AU,Australia,"POLYGON((151.164354085922 -33.9301772341877, 1..."


# Preprocessing

In [19]:
# Remove possible columns with na
df = cat.copy(deep=True)
df = cat.dropna()

In [69]:
# Create a sepeate dataframe with AirportName, City and Country
df_geo = df.copy(deep=True)
df_geo = df[['AirportName','Centroid','City','State','Country']]
# Remove repeated rows (Unique values)
df_geo = df_geo.drop_duplicates(subset=["AirportName"], ignore_index=True)
# Adding new columns for longitudes and latitudes as numbers
df_geo['Long'], df_geo['Lat'] = df_geo['Centroid'].str.split(' ').str
df_geo['Long'] = df_geo['Long'].str[6:].astype(float)
df_geo['Lat'] = df_geo['Lat'].str[:-1].astype(float)
df_geo.head()

<ipython-input-69-ae4ab70c5dfe>:7: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df_geo['Long'], df_geo['Lat'] = df_geo['Centroid'].str.split(' ').str


,AirportName,Centroid,City,State,Country,Long,Lat
0,Kingsford Smith,POINT(151.180087713813 -33.9459774986125),Sydney,New South Wales,Australia,151.180088,-33.945977
1,Santiago International Airport,POINT(-70.7942372739446 -33.3907265266744),Santiago,Santiago Province,Chile,-70.794237,-33.390727
2,Calgary International,POINT(-114.013122872131 51.1184753727795),Calgary,Alberta,Canada,-114.013123,51.118475
3,Edmonton International,POINT(-113.576260685047 53.308783097969),Leduc County,Alberta,Canada,-113.576261,53.308783
4,Vancouver International,POINT(-123.17754124324 49.1935788600694),Richmond,British Columbia,Canada,-123.177541,49.193579


In [61]:
# Sort the data by Date column
df_date = df.sort_values(by=['Date'])
df_date.head()

,AggregationMethod,Date,Version,AirportName,PercentOfBaseline,Centroid,City,State,ISO_3166_2,Country,Geography
302,Daily,2020-03-16,1.0,Santiago International Airport,96,POINT(-70.7942372739446 -33.3907265266744),Santiago,Santiago Province,CL,Chile,"POLYGON((-70.8043956756592 -33.3719683869737, ..."
1713,Daily,2020-03-16,1.0,Halifax International,72,POINT(-63.5116720134583 44.8830168353507),Halifax,Nova Scotia,CA-NS,Canada,"POLYGON((-63.5266613960266 44.8984438169036, -..."
3963,Daily,2020-03-16,1.0,Hartsfield-Jackson Atlanta International,92,POINT(-84.4279188822754 33.6410758198944),College Park,Georgia,US-GA,United States of America (the),"POLYGON((-84.4299745559692 33.657852680999, -8..."
2989,Daily,2020-03-16,1.0,Los Angeles International,98,POINT(-118.404993180627 33.941369379328),Los Angeles,California,US-CA,United States of America (the),"POLYGON((-118.439612388611 33.9517616366508, -..."
5701,Daily,2020-03-16,1.0,McCarran International,68,POINT(-115.14888081287 36.082853976328),Paradise,Nevada,US-NV,United States of America (the),"POLYGON((-115.158487558365 36.1007120735771, -..."


# Data Analysis and Visualization

In [38]:
# Overall Statistics Here:
print("The number of rows of covid_impact_on_airport_traffic dataset is " + str(len(df)) + ".")
airport_names = df['AirportName'].unique()
country_names = df['Country'].unique()
ISO_names = df['ISO_3166_2'].unique()
state_names = df['State'].unique()
city_names = df['City'].unique()
print(airport_names)
print('airport_numbers:' + str(len(airport_names)) + '\n')
print(country_names)
print('country_numbers:' + str(len(country_names)) + '\n')
print(ISO_names)
print('ISO_numbers:' + str(len(ISO_names)) + '\n')
print(state_names)
print('state_numbers:' + str(len(state_names)) + '\n')
print(city_names)
print('city_numbers:' + str(len(city_names)) + '\n')

The number of rows of covid_impact_on_airport_traffic dataset is 7247.
['Kingsford Smith' 'Santiago International Airport'
 'Calgary International' 'Edmonton International'
 'Vancouver International' 'Winnipeg International'
 'Halifax International' 'Toronto Pearson' 'Hamilton International'
 'Montreal Mirabel' 'Montreal Trudeau' 'Los Angeles International'
 'San Francisco International' 'Denver International'
 'Miami International ' 'Hartsfield-Jackson Atlanta International '
 'Daniel K. Inouye International ' 'Chicago OHare International'
 'Boston Logan International ' 'Detroit Metropolitan Wayne County '
 'Charlotte Douglas International' 'Newark Liberty International '
 'McCarran International' 'LaGuardia' 'John F. Kennedy International'
 'Dallas/Fort Worth International ' 'Washington Dulles International '
 'Seattle-Tacoma International ']
airport_numbers:28

['Australia' 'Chile' 'Canada' 'United States of America (the)']
country_numbers:4

['AU' 'CL' 'CA-AB' 'CA-BC' 'CA-MB' 'CA-N

In [80]:
# Geographical Map of all airports
airport_map = folium.Map(location=None)

marker = folium.FeatureGroup()
# Previous overall statistics showed that we have 4 countries: Australia, Chile, Canada and the United States of America
# We label airport belonging to different countries with different colors:
# Green: Australia
# Orange: Chile
# Red: Canada
# Blue: the United States of America
for i in range(len(df_geo)):
    country_color = 'blue'
    if df_geo.loc[i, 'Country'] == 'Australia':
        country_color = 'green'
    elif df_geo.loc[i, 'Country'] == 'Chile':
        country_color = 'orange'
    elif df_geo.loc[i, 'Country'] == 'Canada':
        country_color = 'red'
    
    marker.add_child(
        folium.Marker(
            location = [df_geo.loc[i, 'Lat'], df_geo.loc[i, 'Long']],
            popup = '[AirportName]: ' + df_geo.loc[i, 'AirportName'] +
            '. \n [Country]: ' + df_geo.loc[i, 'Country'] +
            '. \n [State]: ' + df_geo.loc[i, 'State'] +
            '. \n [City]: ' + df_geo.loc[i, 'City'] + '.',
            icon = folium.Icon(color=country_color, icon='plane')
        )
    )

airport_map.add_child(marker)
airport_map.add_child(folium.map.LayerControl())
airport_map

In [74]:
map